# **Setup**

In [1]:
import pandas as pd
import numpy as np

# **Import the Data**
All imported data was retrieved from the Nespresso Canada, USA, UK, and Australia Websites.

In [2]:
dfVertuo_Raw = pd.read_excel("../data/CoffeeData.xlsx", index_col=False, sheet_name="Vertuo");
dfVertuo_Raw = dfVertuo_Raw[dfVertuo_Raw["Status"] == "Current"];

dfOriginal_Raw = pd.read_excel("../data/CoffeeData.xlsx", index_col=False, sheet_name="Original");
dfOriginal_Raw = dfOriginal_Raw[dfOriginal_Raw["Status"] == "Current"];

df = pd.concat([dfVertuo_Raw, dfOriginal_Raw]).reset_index(drop=True);
df.head(1)

,ID,Name,Type,Serving,Serving Size,Headline,Intensity,Sleeve Price,Per Capsule Price,Caption,...,Creamy Texture,Ingredients & Allergens,Number of Capsules per Sleeve,Net Weight per Total Number of Capsules,Capsule Image Link,Capsule & Sleeve Image Link,Decaf Coffee?,Category,Other Information,Status
0,VL01,Intenso,Vertuo,Coffee,230ml,Smooth & Strong,9.0,12.6,1.26,Why we love it: Try Intenso - a Vertuo coffee ...,...,NaN,Roast and ground coffee,10,125 g,https://www.nespresso.com/ecom/medias/sys_mast...,https://www.nespresso.com/shared_res/agility/n...,No,Signature Coffee,NaN,Current


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 30 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   ID                                       70 non-null     object 
 1   Name                                     70 non-null     object 
 2   Type                                     70 non-null     object 
 3   Serving                                  70 non-null     object 
 4   Serving Size                             70 non-null     object 
 5   Headline                                 70 non-null     object 
 6   Intensity                                55 non-null     float64
 7   Sleeve Price                             70 non-null     float64
 8   Per Capsule Price                        70 non-null     float64
 9   Caption                                  70 non-null     object 
 10  Taste                                    70 non-null

In [4]:
df.columns

Index(['ID', 'Name', 'Type', 'Serving', 'Serving Size', 'Headline',
       'Intensity', 'Sleeve Price', 'Per Capsule Price', 'Caption', 'Taste',
       'Best Served As', 'Notes', 'Acidity', 'Bitterness', 'Roastness', 'Body',
       'Milky Taste', 'Bitterness with Milk', 'Roastiness with Milk',
       'Creamy Texture', 'Ingredients & Allergens',
       'Number of Capsules per Sleeve',
       'Net Weight per Total Number of Capsules', 'Capsule Image Link',
       'Capsule & Sleeve Image Link', 'Decaf Coffee?', 'Category',
       'Other Information', 'Status'],
      dtype='object')

# **Data Cleaning**
As the retrieved data contains NULL values, due to the information not being available on the Nespresso website/s, I have attempted to address them in this step. I also consulted my team leaders at the Metrotown boutique to provide their own interpretation regarding the intensities of certain coffee flavours that do not have a numerical value. Specifically, I consulted the Bard Team Lead (Ms. Kashish Bhandari) and the Total Quality Management Team Lead (Ms. Zoe Jia). They are both highly regarded and respected professionals at the Nespresso Metrotown branch by both Management and Coffee Specialists alike. In addition, I have also tried to derive textual meaning from the numerical column values so that they can be incorporated in the NLP pre-processing step. I also consulted my Team Leads regarding this as well, and asked them to attempt to classify the taste profile levels that typically utilize a 1 to 5 scale.

In [5]:
# HANDLE NULL INTENSITY COLUMN VALUES
# I had to do research using external sources, which could be based on past versions of flavors or opinions, as well as apply my own judgement to determine the intensity levels of coffee flavors that are not available. At least as of June 2021, coffee of Barista Creations category do not have an intensity level provided but are generally considered a medium roast. As of September 2021, coffee of Craft Brew category do not have an intensity level by default; they are generally medium roast but the 'Carafe Pour-Over Style Mild' is a more blonder roast than the regular 'Carafe Pour-Over Style'.

nullIntensityRowIDs = df[df['Intensity'].isnull()].index.to_list();
for id in nullIntensityRowIDs:
    # VERTUO -----------------------
    # Carafe Pour-Over Style Mild; My Guess = 5, Team Lead Guess = 3 or 4
    # Carafe Pour-Over Style; My Guess = 7, Team Lead Guess = 7
    # Bianco Forte; My Guess = 7, Team Lead Guess = 7 or 8
    # Bianco Piccolo; My Guess = 6, Team Lead Guess = 6 or 7
    if df.loc[id, 'Name'] == "Carafe Pour-Over Style Mild":
        df.loc[id, 'Intensity'] = 5;
    elif (df.loc[id, 'Name'] == "Carafe Pour-Over Style") | (df.loc[id, 'Name'] == "Bianco Forte") | (df.loc[id, 'Name'] == 'Bianco Piccolo'):
        df.loc[id, 'Intensity'] = 7;
    # ORIGINAL --------------------
    # Corto; My Guess = 11, Team Lead Guess = 7 or 8
    elif (df.loc[id, "Name"] == "Corto"):
        df.loc[id, "Intensity"] = 11;
    # Scuro = 8
    elif (df.loc[id, "Name"] == "Scuro"):
        df.loc[id, "Intensity"] = 8;
    # ALL OTHER REMAINING VERTUO & ORIGINAL FLAVOURS
    # VERTUO:
    # Vanilla Custard Pie = 6
    # Caramel Cookie = 6
    # Hazelino Muffin = 6
    # Chocololate Fudge = 6
    # Bianco Leggero = 6 (Guess)
    # ORIGINAL:
    # Chiaro = 6
    # Cocoa Truffle = 6
    # Vanilla Eclair = 6
    # Caramel Creme Brulle = 6
    else:
        df.loc[id, 'Intensity'] = 6;

In [6]:
# INDICATE WHICH COFFEES DO NOT HAVE AN OFFICIAL INTENSITY LEVEL
# From the above cell, it can be seen that some coffee flavors do not have an official intensity level provided by Nespresso. In turn, an estimate is provided to accomodate for null values. In turn, it is important to indicate in the 'Other Information' column which coffee flavors have an estimated intensity level for the purpose of transparency & clarity.

message = "The intensity provided is an estimate as this coffee does not have an official intensity level";
for id in nullIntensityRowIDs:
    if df.loc[id, "Category"] == "Craft Brew":
        df.loc[id, "Other Information"] += f", {message}";
    else:
        df.loc[id, "Other Information"] = message;

In [7]:
# HANDLE NULL ACIDITY, BITTERNESS, ROASTNESS, AND BODY VALUES
# All null values are of category 'Barista Creations', so will set them to 3 as it respective to the middle of the scale between 1 to 5.

if (df[df['Acidity'].isnull()].index.tolist() == df[df['Bitterness'].isnull()].index.tolist() == df[df['Roastness'].isnull()].index.tolist() == df[df['Body'].isnull().tolist()].index.tolist()) == True:
    nullRowIDs = df[df['Acidity'].isnull()].index.tolist();
    df.loc[nullRowIDs, ['Acidity','Bitterness','Roastness','Body']] = 3;
else:
    print("Invalid. Handle null values manually.")

In [8]:
# HANDLE NULL MILKY TASTE, BITTERNESS WITH MILK, ROASTINESS WITH MILK, AND CREAMY TEXTURE VALUES
# All null values are of all other categories besides 'Barista Creations', so will set them to 3 as it respective to the middle of the scale between 1 to 5

if (df[df['Milky Taste'].isnull()].index.tolist() == df[df['Bitterness with Milk'].isnull()].index.tolist() == df[df['Roastiness with Milk'].isnull()].index.tolist() == df[df['Creamy Texture'].isnull().tolist()].index.tolist()) == True:
    nullRowIDs = df[df['Milky Taste'].isnull()].index.tolist();
    df.loc[nullRowIDs, ['Milky Taste','Bitterness with Milk','Roastiness with Milk','Creamy Texture']] = 3;
else:
    print("Invalid. Handle null values manually.")

In [9]:
# Light Roast: 1 to 4
# Medium Roast: 5 to 8
# Dark Roast: 9 to 13

def determineRoastType(intensity):
    if (intensity > 0) & (intensity < 5):
        return "Blonde";
    elif (intensity >= 5) & (intensity <= 8):
        return "Medium";
    else:
        return "Dark";
df["Roast Type"] = df.apply(lambda x: determineRoastType(x['Intensity']), axis=1);

In [10]:
# Light Roast: 1 to 4 -> Low
# Medium Roast: 5 to 8 -> Medium
# Dark Roast: 9 to 13 -> High

def determineIntensityClassification(intensity):
    if (intensity > 0) & (intensity < 5):
        return "Low";
    elif (intensity >= 5) & (intensity <= 8):
        return "Medium";
    else:
        return "High";
df["Intensity Classification"] = df.apply(lambda x: determineIntensityClassification(x['Intensity']), axis=1);

In [11]:
# My Classification of Taste Profile Level
# Low Taste Profile Level = 1 to 2
# Medium Taste Profile Level = 3 to 4
# High Taste Profile Level = 5

# Team Lead Classification of Taste Profile Level
# Low Taste Profile Level = 1
# Medium Taste Profile Level = 2 to 3
# High Taste Profile Level = 4 to 5

def determineTasteProfileClassification(tasteProfileValue):
    if (tasteProfileValue > 0) & (tasteProfileValue < 2):
        return "Low";
    elif (tasteProfileValue >= 2) & (tasteProfileValue <= 3):
        return "Medium";
    else:
        return "High";
        
for col in ['Acidity','Bitterness','Roastness','Body']:
    df[col + " Classification"] = df.apply(lambda x: determineTasteProfileClassification(x[col]), axis=1);

In [12]:
# My Classification of Taste Profile with Milk Level
# Low Taste Profile Level = 1 to 2
# Medium Taste Profile Level = 3 to 4
# High Taste Profile Level = 5

# Team Lead Classification of Taste Profile with Milk Level
# Low Taste Profile with Milk Level = 1
# Medium Taste Profile with Milk Level = 2 to 3
# High Taste Profile with Milk Level = 4 to 5

def determineTasteProfileWithMilkClassification(tasteProfileWithMilkValue):
    if (tasteProfileWithMilkValue > 0) & (tasteProfileWithMilkValue < 2):
        return "Low";
    elif (tasteProfileWithMilkValue >= 2) & (tasteProfileWithMilkValue <= 3):
        return "Medium";
    else:
        return "High";
for col in ['Milky Taste', 'Bitterness with Milk', 'Roastiness with Milk',
'Creamy Texture']:
    df[col + " Classification"] = df.apply(lambda x: determineTasteProfileWithMilkClassification(x[col]), axis=1);

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 40 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   ID                                       70 non-null     object 
 1   Name                                     70 non-null     object 
 2   Type                                     70 non-null     object 
 3   Serving                                  70 non-null     object 
 4   Serving Size                             70 non-null     object 
 5   Headline                                 70 non-null     object 
 6   Intensity                                70 non-null     float64
 7   Sleeve Price                             70 non-null     float64
 8   Per Capsule Price                        70 non-null     float64
 9   Caption                                  70 non-null     object 
 10  Taste                                    70 non-null

In [14]:
df.columns

Index(['ID', 'Name', 'Type', 'Serving', 'Serving Size', 'Headline',
       'Intensity', 'Sleeve Price', 'Per Capsule Price', 'Caption', 'Taste',
       'Best Served As', 'Notes', 'Acidity', 'Bitterness', 'Roastness', 'Body',
       'Milky Taste', 'Bitterness with Milk', 'Roastiness with Milk',
       'Creamy Texture', 'Ingredients & Allergens',
       'Number of Capsules per Sleeve',
       'Net Weight per Total Number of Capsules', 'Capsule Image Link',
       'Capsule & Sleeve Image Link', 'Decaf Coffee?', 'Category',
       'Other Information', 'Status', 'Roast Type', 'Intensity Classification',
       'Acidity Classification', 'Bitterness Classification',
       'Roastness Classification', 'Body Classification',
       'Milky Taste Classification', 'Bitterness with Milk Classification',
       'Roastiness with Milk Classification', 'Creamy Texture Classification'],
      dtype='object')

# **Export Cleaned Data**

In [15]:
df.to_csv('../data/CleanedCoffeeData.csv', index=False);